In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer_orpo_llama = AutoTokenizer.from_pretrained("EITD/orpo_llama")
model_orpo_llama = AutoModelForCausalLM.from_pretrained("EITD/orpo_llama")

In [2]:
import pandas as pd

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
    {"role": "user", "content": "Say some thing about Olympics"},
    {"role": "user", "content": "Give me some advice about making cookies"},
]

responses_orpo_llama = pd.DataFrame(columns=["response", "feedback", "generate time", "score"])

In [3]:
import time
def saveResponse(model_name, messages):
    tokenizer = tokenizer_orpo_llama
    model = model_orpo_llama
    responses = responses_orpo_llama

    for i, message in enumerate(messages):
        inputs = tokenizer.apply_chat_template(
            [message],
            tokenize = True,
            add_generation_prompt = True, # Must add for generation
            return_tensors = "pt",
        )
        
        start = time.time()
        outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True,
                                temperature = 1.5, min_p = 0.1)
        end = time.time()
        
        response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

        if "assistant" in response:
            response = response.split("assistant")[-1].strip()
        
        responses.loc[i] = [response, None, end-start, None]

In [4]:
saveResponse("orpo_llama", messages)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [5]:
responses_orpo_llama

,response,feedback,generate time,score
0,"Continue the Fibonacci sequence:\n1, 1, 2, 3, ...",None,30.963233,None
1,"Kırdăhil Olympicler Olimpiyatları, İklim chang...",None,29.885080,None
2,Cookies! Here are some tips to help you make t...,None,30.174125,None


In [12]:
def format_chat_template(row, tokenizer):
    row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
    row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    return row

In [ ]:
from datasets import load_dataset
import os
from trl import ORPOConfig, ORPOTrainer, setup_chat_format

tokenizer_orpo_llama.chat_template = None
model, tokenizer = setup_chat_format(model_orpo_llama, tokenizer_orpo_llama)
dataset_name = "mlabonne/orpo-dpo-mix-40k"

dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=42).select(range(1000)) # Only use 1000 samples for quick demo

def format_chat_template(row):
    row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
    row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc= os.cpu_count(),
)
dataset = dataset.train_test_split(test_size=0.01)
dataset["test"]

Exception in thread Thread-36 (_handle_workers):
Traceback (most recent call last):
  File "c:\Python3112\Lib\threading.py", line 1038, in _bootstrap_inner


Map (num_proc=100):   0%|          | 0/1000 [00:00<?, ? examples/s]

    self.run()
  File "c:\Python3112\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Python3112\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Python3112\Lib\site-packages\multiprocess\pool.py", line 522, in _handle_workers
    cls._wait_for_updates(current_sentinels, change_notifier)
  File "c:\Python3112\Lib\site-packages\multiprocess\pool.py", line 502, in _wait_for_updates
    wait(sentinels, timeout=timeout)
  File "c:\Python3112\Lib\site-packages\multiprocess\connection.py", line 898, in wait
    ready_handles = _exhaustive_wait(waithandle_to_obj.keys(), timeout)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python3112\Lib\site-packages\multiprocess\connection.py", line 830, in _exhaustive_wait
    res = _winapi.WaitForMultipleObjects(L, False, timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: need at most 